In [1]:
import tensorflow as tf 

# Introduction

Looking at classifying digits with Neural Networks. TODO: Explain MNIST a bit here 

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Network Creation

First, define some parameters

In [46]:
numClasses = 10
inputSize = 784  
numHiddenUnits = 50
trainingIterations = 10000
batchSize = 100

Just in case

In [47]:
tf.reset_default_graph() 

Start with placeholders

In [48]:
X = tf.placeholder(tf.float32, shape = [None, inputSize])
y = tf.placeholder(tf.float32, shape = [None, numClasses])

Next, onto the weight matrices and bias terms

In [49]:
#W1 = tf.Variable(tf.constant(0.01, shape = [inputSize, numHiddenUnits]))
#B1 = tf.Variable(tf.constant(0.01), [numHiddenUnits])
#W2 = tf.Variable(tf.constant(0.01, shape = [numHiddenUnits, numClasses]))
#B2 = tf.Variable(tf.constant(0.01), [numClasses])

# Initialization is so crazy important with neural networks. The above didn't
# converge, cuz the values got way too high, so I guess pulling from a normal 
# distribution instead of constants for initialization is the way to go. 

W1 = tf.Variable(tf.truncated_normal([inputSize, numHiddenUnits], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits])
W2 = tf.Variable(tf.truncated_normal([numHiddenUnits, numClasses], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numClasses])

In [50]:
hiddenLayerOutput = tf.matmul(X, W1) + B1
hiddenLayerOutput = tf.nn.relu(hiddenLayerOutput)
finalOutput = tf.matmul(hiddenLayerOutput, W2) + B2
finalOutput = tf.nn.relu(finalOutput)

# Loss Function

In [51]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = finalOutput))
opt = tf.train.GradientDescentOptimizer(learning_rate = .1).minimize(loss)


The following are the statements that help with calculating accuracy

In [52]:
correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Training

In [53]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(trainingIterations):
    batch = mnist.train.next_batch(batchSize)
    batchInput = batch[0]
    batchLabels = batch[1]
    _, trainingLoss = sess.run([opt, loss], feed_dict={X: batchInput, y: batchLabels})
    if i%1000 == 0:
        trainAccuracy = accuracy.eval(session=sess, feed_dict={X: batchInput, y: batchLabels})
        #train_loss = loss.eval(session=sess, feed_dict={X: batchInput, y: batchLabels})
        print ("step %d, training accuracy %g"%(i, trainAccuracy))
        #print ("step %d, training loss %g"%(i, train_loss))

step 0, training accuracy 0.11
step 1000, training accuracy 0.93
step 2000, training accuracy 0.96
step 3000, training accuracy 0.97
step 4000, training accuracy 0.97
step 5000, training accuracy 0.98
step 6000, training accuracy 0.97
step 7000, training accuracy 0.98
step 8000, training accuracy 0.99
step 9000, training accuracy 1


# Testing

In [54]:
batch = mnist.test.next_batch(batchSize)
testLoss = sess.run(accuracy, feed_dict={X: batch[0], y: batch[1]})
print ("test accuracy %g"%(testLoss))

test accuracy 0.98


# 2 Layer Neural Net

In [45]:
numHiddenUnitsLayer2 = 100
trainingIterations = 10000
tf.reset_default_graph() 

X = tf.placeholder(tf.float32, shape = [None, inputSize])
y = tf.placeholder(tf.float32, shape = [None, numClasses])

W1 = tf.Variable(tf.random_normal([inputSize, numHiddenUnits], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits])
W2 = tf.Variable(tf.random_normal([numHiddenUnits, numHiddenUnitsLayer2], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numHiddenUnitsLayer2])
W3 = tf.Variable(tf.random_normal([numHiddenUnitsLayer2, numClasses], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1), [numClasses])

hiddenLayerOutput = tf.matmul(X, W1) + B1
hiddenLayerOutput = tf.nn.relu(hiddenLayerOutput)
hiddenLayer2Output = tf.matmul(hiddenLayerOutput, W2) + B2
hiddenLayer2Output = tf.nn.relu(hiddenLayer2Output)
finalOutput = tf.matmul(hiddenLayer2Output, W3) + B3

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = finalOutput))
opt = tf.train.GradientDescentOptimizer(learning_rate = .1).minimize(loss)

correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(trainingIterations):
    batch = mnist.train.next_batch(batchSize)
    batchInput = batch[0]
    batchLabels = batch[1]
    _, trainingLoss = sess.run([opt, loss], feed_dict={X: batchInput, y: batchLabels})
    if i%1000 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X: batchInput, y: batchLabels})
        print ("step %d, training accuracy %g"%(i, train_accuracy))

batch = mnist.test.next_batch(batchSize)
testAccuracy = sess.run(accuracy, feed_dict={X: batch[0], y: batch[1]})
print ("test accuracy %g"%(testAccuracy))

step 0, training accuracy 0.06
step 1000, training accuracy 0.95
step 2000, training accuracy 1
step 3000, training accuracy 1
step 4000, training accuracy 0.96
step 5000, training accuracy 0.98
step 6000, training accuracy 1
step 7000, training accuracy 1
step 8000, training accuracy 0.99
step 9000, training accuracy 1
test accuracy 0.97


# Exercises

* Definitely something with changing the number of layers and seeing the corresponding changes in accuracy
* Maybe trying this approach with the Boston dataset since you've gone over that before?


# Additional Resources

* Stanford's Neural Network [Tutorial](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/)
* Michael Neilson's [Online Book](http://neuralnetworksanddeeplearning.com/)
* Andrej Karpathy's [Blog Post](http://karpathy.github.io/neuralnets/ )